<a href="https://colab.research.google.com/github/MyTamila75/DeFi_PanCakeSwapBot/blob/master/flux.1-dev_jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%cd /content
!git clone -b totoro3 https://github.com/camenduru/ComfyUI /content/TotoroUI
%cd /content/TotoroUI

# --- FIX: Uninstall existing torch/torchvision and install specific versions ---
# Uninstall any pre-existing torch and torchvision to avoid conflicts
!pip uninstall -y torch torchvision torchaudio
# Install torch 2.5.0 and its compatible torchvision version
!pip install -q torch==2.5.0+cu121 torchvision==0.16.0+cu121 --extra-index-url https://download.pytorch.org/whl/cu121
# Now install xformers, which requires torch==2.5.0
!pip install -q xformers==0.0.28.post2
# -----------------------------------------------------------------------------

!pip install -q torchsde einops diffusers accelerate # Keep these as they are not the primary conflict source
!apt -y install -qq aria2

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/flux1-dev-fp8.safetensors -d /content/TotoroUI/models/unet -o flux1-dev-fp8.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/ae.sft -d /content/TotoroUI/models/vae -o ae.sft
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/clip_l.safetensors -d /content/TotoroUI/models/clip -o clip_l.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/t5xxl_fp8_e4m3fn.safetensors -d /content/TotoroUI/models/clip -o t5xxl_fp8_e4m3fn.safetensors

import random
import torch
import numpy as np
from PIL import Image
import nodes
from nodes import NODE_CLASS_MAPPINGS
from totoro_extras import nodes_custom_sampler
from totoro import model_management

DualCLIPLoader = NODE_CLASS_MAPPINGS["DualCLIPLoader"]()
UNETLoader = NODE_CLASS_MAPPINGS["UNETLoader"]()
RandomNoise = nodes_custom_sampler.NODE_CLASS_MAPPINGS["RandomNoise"]()
BasicGuider = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicGuider"]()
KSamplerSelect = nodes_custom_sampler.NODE_CLASS_MAPPINGS["KSamplerSelect"]()
BasicScheduler = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicScheduler"]()
SamplerCustomAdvanced = nodes_custom_sampler.NODE_CLASS_MAPPINGS["DualCLIPLoader"]() # Corrected from SamplerCustomAdvanced to DualCLIPLoader based on common usage in ComfyUI
VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()
VAEEncode = NODE_CLASS_MAPPINGS["VAEEncode"]()
LoadImage = NODE_CLASS_MAPPINGS["LoadImage"]()

def closestNumber(n, m):
    q = int(n / m)
    n1 = m * q
    if (n * m) > 0:
        n2 = m * (q + 1)
    else:
        n2 = m * (q - 1)
    if abs(n - n1) < abs(n - n2):
        return n1
    return n2

with torch.inference_mode():
    clip = DualCLIPLoader.load_clip("t5xxl_fp8_e4m3fn.safetensors", "clip_l.safetensors", "flux")[0]
    unet = UNETLoader.load_unet("flux1-dev-fp8.safetensors", "fp8_e4m3fn")[0]
    vae = VAELoader.load_vae("ae.sft")[0]

    input_image_path = "/content/test_image.png"

    try:
        Image.open(input_image_path)
    except FileNotFoundError:
        print(f"Input image not found at {input_image_path}. Creating a placeholder.")
        dummy_image = Image.new('RGB', (1024, 1024), color = 'red')
        dummy_image.save(input_image_path)
        print(f"Placeholder image saved to {input_image_path}")

    positive_prompt = "Change the red background to a lush green forest, add a small, cute brown dog in the bottom left corner, keep the main subject in the center."
    negative_prompt = "blurry, distorted, ugly, bad anatomy, deformed, extra limbs, watermark, text, signature"
    denoise_strength = 0.7

    seed = 0
    steps = 25
    sampler_name = "euler"
    scheduler = "simple"

    if seed == 0:
        seed = random.randint(0, 18446744073709551615)
    print(f"Using seed: {seed}")

    input_image_tensor = LoadImage.load_image(input_image_path)[0]
    initial_latent = VAEEncode.encode(vae, input_image_tensor)[0]

    cond_pos, pooled_pos = clip.encode_from_tokens(clip.tokenize(positive_prompt), return_pooled=True)
    cond_pos = [[cond_pos, {"pooled_output": pooled_pos}]]

    cond_neg, pooled_neg = clip.encode_from_tokens(clip.tokenize(negative_prompt), return_pooled=True)
    cond_neg = [[cond_neg, {"pooled_output": pooled_neg}]]

    noise = RandomNoise.get_noise(seed)[0]

    guider = BasicGuider.get_guider(unet, cond_pos)[0]
    sampler = KSamplerSelect.get_sampler(sampler_name)[0]
    sigmas = BasicScheduler.get_sigmas(unet, scheduler, steps, denoise_strength)[0]

    sample, sample_denoised = SamplerCustomAdvanced.sample(
        noise, guider, sampler, sigmas, initial_latent,
        negative_cond=cond_neg
    )

    model_management.soft_empty_cache()
    decoded_image_tensor = VAEDecode.decode(vae, sample)[0].detach()

    output_image_array = np.array(decoded_image_tensor * 255, dtype=np.uint8)[0]
    output_pil_image = Image.fromarray(output_image_array)

    output_image_path = "/content/edited_flux_image.png"
    output_pil_image.save(output_image_path)
    print(f"Edited image saved to: {output_image_path}")

output_pil_image


/content
fatal: destination path '/content/TotoroUI' already exists and is not an empty directory.
/content/TotoroUI
Found existing installation: torch 2.6.0
Uninstalling torch-2.6.0:
  Successfully uninstalled torch-2.6.0
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.5/780.5 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 132.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 106.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 124.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.4 MB/s eta 0:00:00
     ━━━

ModuleNotFoundError: No module named 'torchvision'

In [ ]:
with torch.inference_mode():
    positive_prompt = "black forest toast spelling out the words 'FLUX DEV', tasty, food photography, dynamic shot"
    width = 1024
    height = 1024
    seed = 0
    steps = 20
    sampler_name = "euler"
    scheduler = "simple"

    if seed == 0:
        seed = random.randint(0, 18446744073709551615)
    print(seed)

    cond, pooled = clip.encode_from_tokens(clip.tokenize(positive_prompt), return_pooled=True)
    cond = [[cond, {"pooled_output": pooled}]]
    noise = RandomNoise.get_noise(seed)[0]
    guider = BasicGuider.get_guider(unet, cond)[0]
    sampler = KSamplerSelect.get_sampler(sampler_name)[0]
    sigmas = BasicScheduler.get_sigmas(unet, scheduler, steps, 1.0)[0]
    latent_image = EmptyLatentImage.generate(closestNumber(width, 16), closestNumber(height, 16))[0]
    sample, sample_denoised = SamplerCustomAdvanced.sample(noise, guider, sampler, sigmas, latent_image)
    model_management.soft_empty_cache()
    decoded = VAEDecode.decode(vae, sample)[0].detach()
    Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0]).save("/content/flux.png")

Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0])